# RAG-based DBMS Assistant using Ollama

## Overview

This project implements a Retrieval-Augmented Generation (RAG) assistant using DBMS notes.

The system retrieves relevant information from PDF notes and generates answers using a local LLM (Ollama).

This approach improves answer accuracy by combining document retrieval and language generation.

---

## Objectives

- Load DBMS notes from PDF
- Split text into chunks
- Store embeddings in vector database
- Retrieve relevant content
- Generate answers using Ollama
- Compare chunk sizes
- Compare prompt engineering strategies

In [1]:
!pip install langchain
!pip install langchain-community
!pip install langchain-text-splitters
!pip install chromadb
!pip install sentence-transformers
!pip install pypdf

In [4]:
# Import required 
import os

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

from langchain_community.embeddings import SentenceTransformerEmbeddings

# Ollama LLM
from langchain_community.llms import Ollama



# Step 1: Load PDF Document

In this step, we load DBMS notes PDF.

This PDF will act as the knowledge source for our assistant.

In [5]:
documents = []

data_folder = "data"

for file in os.listdir(data_folder):
    
    if file.endswith(".pdf"):
        
        loader = PyPDFLoader(os.path.join(data_folder, file))
        
        documents.extend(loader.load())

print("Total pages loaded:", len(documents))

Total pages loaded: 668


# Step 2: Create Small Chunks (Experiment 1)

We split the document into small chunks.

Chunk size = 500 characters

Small chunks help retrieve more precise information.

In [6]:
# Import text splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Create small chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

# Split document
chunks = text_splitter.split_documents(documents)

print("Small chunks created:", len(chunks))

Small chunks created: 2308


# Step 3: Create Large Chunks (Experiment 2)

Now we create larger chunks.

Chunk size = 1000 characters

Large chunks provide more context but may reduce retrieval precision.

In [7]:
# Create large chunks
text_splitter_large = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

chunks_large = text_splitter_large.split_documents(documents)

print("Large chunks created:", len(chunks_large))

Large chunks created: 1274


In [8]:
# Import embedding model
from langchain_community.embeddings import SentenceTransformerEmbeddings

# Load embedding model
embedding = SentenceTransformerEmbeddings(
    model_name="all-MiniLM-L6-v2"
)

print("Embeddings ready")

C:\Users\kalpa\AppData\Local\Temp\ipykernel_10044\1112582225.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding = SentenceTransformerEmbeddings(
C:\Users\kalpa\anaconda3\envs\rag_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\kalpa\anaconda3\envs\rag_env\lib\site-packages\huggingface_hub\file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_down

Embeddings ready


In [21]:
from langchain_community.vectorstores import Chroma

# Create vector database for small chunks
vectorstore_small = Chroma.from_documents(
    documents=chunks,
    embedding=embedding,
    persist_directory="./db_small"
)

print("Small chunk vector DB created")

Small chunk vector DB created


In [10]:
# Create vector database for large chunks
vectorstore_large = Chroma.from_documents(
    documents=chunks_large,
    embedding=embedding,
    persist_directory="./db_large"
)

print("Large chunk vector DB created")

Large chunk vector DB created


# Step 6: Create Retriever

Retriever finds relevant chunks based on user question.

In [22]:
# Retriever for small chunks
retriever_small = vectorstore_small.as_retriever(search_kwargs={"k": 3})

# Retriever for large chunks
retriever_large = vectorstore_large.as_retriever(search_kwargs={"k": 3})

print("Retrievers ready")

Retrievers ready


# Step 7: Load Ollama Model

We use Ollama to generate answers locally.


In [13]:
from langchain_community.llms import Ollama

# Connect to Ollama model
llm = Ollama(model="gemma3:1b")

print("Ollama connected")

Ollama connected


# Step 8: Small Chunk Assistant

In [23]:
def ask_small_chunks(question):

    docs = retriever_small.invoke(question)

    context = "\n".join([doc.page_content for doc in docs])

    prompt = f"""
Answer using only the context below.

Context:
{context}

Question:
{question}

Answer:
"""

    answer = llm.invoke(prompt)

    return answer

# Step 9: Large Chunk Assistant

In [24]:
def ask_large_chunks(question):

    docs = retriever_large.invoke(question)

    context = "\n".join([doc.page_content for doc in docs])

    prompt = f"""
Answer using only the context below.

Context:
{context}

Question:
{question}

Answer:
"""

    answer = llm.invoke(prompt)

    return answer

# Step 10: Basic Prompt Assistant

This assistant uses a simple prompt.

This will be used for prompt engineering comparison.

In [14]:
def ask_dbms_assistant_basic(question):

    # Retrieve relevant chunks
    docs = retriever.invoke(question)

    # Combine retrieved text
    context = "\n".join([doc.page_content for doc in docs])

    # Basic prompt
    prompt = f"""
Context:
{context}

Question:
{question}

Answer:
"""

    # Generate answer
    answer = llm.invoke(prompt)

    return answer

# Step 11: Improved Prompt Assistant

This assistant uses better prompt instructions.

This improves answer quality.

In [27]:
def ask_dbms_assistant_improved(question):

    docs = retriever.invoke(question)

    context = "\n".join([doc.page_content for doc in docs])

    # Improved prompt
    prompt = f"""
You are a DBMS expert teacher.

Explain clearly in simple and detailed way.

Context:
{context}

Question:
{question}

Detailed Answer:
"""

    answer = llm.invoke(prompt)

    return answer

# Step 12: Test Questions

In [30]:
questions = [
    "What is normalization?",
    "What is primary key?",
    "Explain ACID properties",
    "What is indexing?"
]

for q in questions:

 
    print("Question:", q)

    print("\nSmall Chunk Answer:")
    print(ask_small_chunks(q))

    print("\nLarge Chunk Answer:")
    print(ask_large_chunks(q))

    print("\nBasic Prompt Answer:")
    print(ask_dbms_assistant_basic(q))

    print("\nImproved Prompt Answer:")
    print(ask_dbms_assistant_improved(q))


Question: What is normalization?

Small Chunk Answer:
Normalization is the process of organizing the attributes of the database to reduce or eliminate data redundancy (having the same data but at different places).

Large Chunk Answer:
Normalization is a technique of organizing the data in the database. It is a systematic approach of decomposing tables to eliminate data redundancy and undesirable characteristics like Insertion, Update and Deletion Anomalies. It helps in eliminating redundant data, ensuring data dependencies make sense, and simplifying the process of managing data.

Basic Prompt Answer:
Normalization is the process of organizing the attributes of a database to reduce or eliminate data redundancy (having the same data but at different places). It helps to eliminate anomalies and improve the overall quality of the database.


Improved Prompt Answer:
Okay, let’s break down normalization in a way that’s easy to understand. Think of it like organizing a really messy library 

# Experiment Results: Chunk Comparison

Small chunks provided more accurate and precise answers.

Large chunks provided more context but sometimes less precise retrieval.

Conclusion: Small chunks performed better.

# Experiment Results: Prompt Engineering

Improved prompt provided clearer and more detailed answers.

Prompt engineering improves answer quality.

# Final Conclusion

This project successfully implemented a RAG-based DBMS assistant.

The system retrieved relevant information and generated correct answers.

Experiments showed that:

• Small chunks improve retrieval accuracy  
• Prompt engineering improves answer quality  

This demonstrates effectiveness of RAG systems.